In [7]:
import os, re, csv
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm
import gensim
import multiprocessing

In [4]:
#corpus 읽어서 태그달기(태그는 임의의 태그, 입력된 순서대로 달림)
def read_corpus(noun_content):
    for i, content in tqdm(enumerate(noun_content)):
        yield gensim.models.doc2vec.TaggedDocument(content, [i])

In [21]:
df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\Q&A.csv")
lem = df["LEMMA"].tolist()
print(lem[:10])
lem_cont = []
for k in lem:
    lem_cont.append(str(k).split())
print(lem_cont[:10])

['이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시', '재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사항 이전 성적 연결 현황 동일 교과 자동연결되었습 나오 학사 공지 나오 대로 재수강 이전 성적 선택 재수강 연결 필요 아무것 감사', '수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제', '처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울 방학 2019 부전공 경영 선택 2020년 학기 이중전공 변경 기간 경영 신청 한번 종합 정보 시스템 들어가 사진 내요 신경 변경 기간 경영 신청', '이중전공 바꾸 바꾸 학점 소멸', '등록 휴학 자퇴 경우 이번 학기 등록금 환불 가능', '졸업 유예 기간 동안 학점 취득 가능', '인간 사회 학점 짜리 과목 씨쁠 이거 언어 문학 학점 짜리 페논 과목 재수강 연결 가능', '2019 학기 학기 마치 사정 학점 상태 휴학 전공 이중전공 학점 채운 상황 취득 학점 132 학점 학점 모자라 추가 학기 상황 경우 2020 여름 계절 학기 학점 수강 수료 가능 답변 부탁 드리', '교양 영어 수강 학생 온라인 과제 코드']
[['이중전공', '부전공', '차이', '대하', '이중전공', '학위', '졸업장', '표시', '부전공', '졸업장', '표시'], ['재수강', '연결', '종합', '정보', '시스템', '수업', '수강', '관리', '수강', '과목', '선택', '메뉴', '들어가', '안내', '사항', '이전', '성적', '연결', '현황', '동일', '교과', '자동연결되었습', '나오', '학사', '공지', '나오', '대로', '재수강', '이전', '성적', '선택', '재수강', '연결', '필요', '아무것', '감사'], ['수고', '다르', '아니', '제가', '평점', '학기', '수업', '평점', '올리', '졸업', '학점', '134', '학점', '

In [25]:
corpus = list(read_corpus(lem_cont))
print(corpus[:10])

cores = multiprocessing.cpu_count()

#doc2vec parameters
vector_size = 1000
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = cores #number of parallel processes

#모델 학습, doc2vec
model = gensim.models.doc2vec.Doc2Vec(
    min_count=word_min_count, size=vector_size, alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count
    )

#모델 생성
model.build_vocab(corpus) #모델 훈련 위한 vocab 생성

# Train document vectors!
for epoch in range(10):
	model.train(corpus, total_examples=model.corpus_count, epochs=train_epoch)
	model.alpha -= 0.002 # decrease the learning rate
	model.min_alpha = model.alpha # fix the learning rate, no decay

# To save
model.save(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\doc2vec.model")
model.save_word2vec_format(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\doc2vec.model.word2vec_format", binary=False)


7015it [00:00, 779562.90it/s]
[TaggedDocument(words=['이중전공', '부전공', '차이', '대하', '이중전공', '학위', '졸업장', '표시', '부전공', '졸업장', '표시'], tags=[0]), TaggedDocument(words=['재수강', '연결', '종합', '정보', '시스템', '수업', '수강', '관리', '수강', '과목', '선택', '메뉴', '들어가', '안내', '사항', '이전', '성적', '연결', '현황', '동일', '교과', '자동연결되었습', '나오', '학사', '공지', '나오', '대로', '재수강', '이전', '성적', '선택', '재수강', '연결', '필요', '아무것', '감사'], tags=[1]), TaggedDocument(words=['수고', '다르', '아니', '제가', '평점', '학기', '수업', '평점', '올리', '졸업', '학점', '134', '학점', '초과', '문제'], tags=[2]), TaggedDocument(words=['처음', '이중전공', '신청', '이중전공', '경영', '변경', '시도', '자체', '시험', '떨어지', '변경', '실패', '지나', '겨울', '방학', '2019', '부전공', '경영', '선택', '2020년', '학기', '이중전공', '변경', '기간', '경영', '신청', '한번', '종합', '정보', '시스템', '들어가', '사진', '내요', '신경', '변경', '기간', '경영', '신청'], tags=[3]), TaggedDocument(words=['이중전공', '바꾸', '바꾸', '학점', '소멸'], tags=[4]), TaggedDocument(words=['등록', '휴학', '자퇴', '경우', '이번', '학기', '등록금', '환불', '가능'], tags=[5]), TaggedDocument(words=['졸업', '유예', '기간', '동안

KeyboardInterrupt: 